In [1]:
import openai
import pandas as pd
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema,StructuredOutputParser

In [2]:
load_dotenv()
openai.api_key=os.getenv("basit_key")



In [3]:
chat=ChatOpenAI(
    temperature=0.0, model="gpt-3.5-turbo",api_key=os.getenv("basit_key")
)


In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [5]:
prompt=f"""Translate the text \
that is delimited by triple backticks 
into a style that is "American English \
in a calm and respectful tone".
text: ```Nous espérons que vous allez bien. Nous tenons à vous remercier pour votre confiance en notre produit, et nous sommes désolés d'apprendre que vous rencontrez des problèmes avec votre machine à laver.\
    Nous comprenons à quel point il est essentiel d'avoir un appareil fonctionnel à la maison, en particulier votre machine à laver. Nous voulons résoudre ce problème le plus rapidement possible```
"""


In [6]:
get_completion(prompt=prompt)

'"We hope you\'re doing well. We want to thank you for your trust in our product, and we\'re sorry to hear that you\'re experiencing issues with your washing machine. We understand how important it is to have a functional appliance at home, especially your washing machine. We want to resolve this issue as quickly as possible."'

In [7]:
template_string = """Translate the text \
that is delimited by triple backticks \
in style tto ahat is {style}. \
text: ```{text}```
"""

In [6]:
from langchain.prompts import ChatPromptTemplate
prompt_template=ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [131]:
customer_text="chal bhai jaa yehan say."


In [132]:
text_style="""American English \
in a harsh tone"""

In [133]:
customer_message=prompt_template.format_messages(
    style=text_style,
    text=customer_text
)

In [134]:
customer_responce=chat(customer_message)

In [136]:
customer_responce.content

'Get lost, dude.'

In [138]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!"""


In [139]:
service_style="a ploite tone/ thats speaks ub pirate english"

In [141]:
service_reply=prompt_template.format_messages(
    text=service_reply,
    style=service_style
)

In [143]:
service_responce=chat(service_reply)

In [145]:
service_responce.content

"Arr, me heartie customer, the warranty be not coverin' the cleanin' expenses fer yer galley 'cause 'tis yer fault ye misused yer blender by forgettin' to put the lid on afore startin' the blender. Tough luck, me matey! Fare thee well!"

In [8]:
own_prompt="""I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, and there was an unpleasant odor emitting from the dryer during use. Overall, I found this product lacking in reliability and quality, especially considering its price.\
In conclusion, I would not recommend this hair dryer based on my experience.\
Emily Smith

I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, and there was an unpleasant odor emitting from the dryer during use. Overall, I found this product lacking in reliability and quality, especially considering its price.\
In conclusion, I would not recommend this hair dryer based on my experience.\
Emily Smith
"""

In [8]:
template="A reviews is given by the customer is tripple backslash and \
    you have to extract the main cause of problem in 2-3 words if there is any, along with name and what they purchased in the following Json format\
    id,customername, product purchased, problem with that product\
    ```{text}```\
    " 

In [9]:
own_template=prompt_template.from_template(template=template)
own_template.messages[0].prompt.input_variables

['text']

In [10]:
customer_review=own_template.format_messages(
    text=own_prompt
)

In [11]:
own_responce=chat(customer_review)

In [12]:
print(own_responce.content)

{
  "id": 1,
  "customername": "Emily Smith",
  "product purchased": "cookingset",
  "problem with that product": "inconsistent performance"
}


In [13]:
response_schemas =[
    ResponseSchema(name="id",description="A unique interger"),
ResponseSchema(name="customername",description="name of person who purchased the product"),
ResponseSchema(name="product purchased",description="name of product"),
ResponseSchema(name="problem with that product",description="what really a problem with product")
]



In [14]:
response_schemas

[ResponseSchema(name='id', description='A unique interger', type='string'),
 ResponseSchema(name='customername', description='name of person who purchased the product', type='string'),
 ResponseSchema(name='product purchased', description='name of product', type='string'),
 ResponseSchema(name='problem with that product', description='what really a problem with product', type='string')]

In [28]:
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)
format_instruction=output_parser.get_format_instructions("""he output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\
```json\
[{\
	"id": string  // A unique interger\
	"customername": string  // name of person who purchased the product \
	"product purchased": string  // name of product \
	"problem with that product": string  // what really a problem with product\
},\
    "id": string  // A unique interger\
	"customername":......"]\
```""")

print(format_instruction)



```json
{
	"id": string  // A unique interger
	"customername": string  // name of person who purchased the product
	"product purchased": string  // name of product
	"problem with that product": string  // what really a problem with product
}
```


In [16]:
template_with_json="reviews is given by the customer is tripple backslash and to separate each review start with another line\
    you have to extract the main cause of problem in 2-3 words if there is any, along with name and what they purchased \
       \
    Format the output as JSON with the following keys :\
    id,customername, product purchased, problem with that product\
    ```{text}\
    {format_instruction}\
    \
    " 

In [17]:
chat_prompt_json=ChatPromptTemplate.from_template(template=template_with_json)

In [18]:
chat_prompt_json.messages[0].prompt.input_variables

['format_instruction', 'text']

In [19]:
json_prompt=chat_prompt_json.format_messages(text=own_prompt,format_instruction=format_instruction)

In [20]:
print(json_prompt[0].content)

reviews is given by the customer is tripple backslash and to separate each review start with another line    you have to extract the main cause of problem in 2-3 words if there is any, along with name and what they purchased            Format the output as JSON with the following keys :    id,customername, product purchased, problem with that product    ```I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.The build quality left much to be desired; the handle felt fragile, and there was an unpleasant odor emitting from the dryer during use. Overall, I found this product lacking in reliability and quality, especially considering its price.In conclusion, I would not recommend this hair dryer based on my experience.Emily Smith
    The 

In [21]:
json_prompt2=chat(json_prompt)

In [22]:
print(json_prompt2.content)

```json
{
	"id": "1",
	"customername": "Emily Smith",
	"product purchased": "cookingset",
	"problem with that product": "inconsistent performance, fluctuating heat settings, loud noise, fragile handle, unpleasant odor"
}
```


In [23]:
print(json_prompt2.content)

```json
{
	"id": "1",
	"customername": "Emily Smith",
	"product purchased": "cookingset",
	"problem with that product": "inconsistent performance, fluctuating heat settings, loud noise, fragile handle, unpleasant odor"
}
```


In [24]:
output=output_parser.parse(json_prompt2.content)

In [25]:
output

{'id': '1',
 'customername': 'Emily Smith',
 'product purchased': 'cookingset',
 'problem with that product': 'inconsistent performance, fluctuating heat settings, loud noise, fragile handle, unpleasant odor'}